In [1]:
!pip install rouge-score

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=99064f449eba3dcfb89ffad7c6d2aafc95dcfb23dd2b8c2407ee9aee9371dac5
  Stored in directory: /home/bvangala1/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [2]:
import torch

In [3]:
import pandas as pd

In [4]:
from rouge_score import rouge_scorer

In [5]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [6]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("billsum")

Found cached dataset parquet (/home/bvangala1/.cache/huggingface/datasets/parquet/billsum-35284aeeba767e98/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
})

In [9]:
train = dataset['train']
test =  dataset['ca_test']

In [10]:
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-billsum")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-billsum")

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [11]:
summarizer = pipeline("summarization", model= model, tokenizer= tokenizer, framework="tf")

In [12]:
system_summary_single = summarizer(train[0]['text'], min_length=200, max_length=500)

In [13]:
standard_summary_single = train[0]['summary']
system_summary_single = system_summary_single[0]['summary_text']

In [14]:
scorer_single = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [16]:
scores_single = scorer_single.score(system_summary_single, standard_summary_single)

In [20]:
pd.DataFrame(scores_single).set_index([['precision','recall','fmeasure']])

,rouge1,rougeL
precision,0.710526,0.672932
recall,0.984375,0.932292
fmeasure,0.825328,0.781659


In [10]:
# total test set
data = dataset['ca_test']
CasesText = data['text']
GoldSummary = data['summary']

SystemSummary = []

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/pegasus-billsum"

pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [12]:
%%time
for i, case in enumerate(CasesText):
    strtolist = []
    strtolist.append(case)

    batch = pegasus_tokenizer(strtolist, truncation=True, padding="longest", return_tensors="pt").to(device)
    summary = pegasus_model.generate(**batch)
    summary_final = pegasus_tokenizer.batch_decode(summary, skip_special_tokens=False)
    SystemSummary.append(summary_final)

    print(i)
    strtolist.clear()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
SystemSummaryFinal = []

In [14]:
for i in SystemSummary:
    SystemSummaryFinal.append((i[0]))

In [15]:
Summaries = pd.DataFrame(list(zip(GoldSummary, SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [16]:
Summaries.to_csv("PegsusSummaries.csv", header=True, index=False)

In [17]:
Summaries.sample(5)

,GoldSummary,SystemSummary
970,Existing law requires law enforcement agencies...,This bill requires a law enforcement agency th...
734,Existing law establishes a process by which a ...,(Sec. 1) Declares that if the parking penalty ...
518,"Existing law, the Swimming Pool Safety Act, pr...",(This measure has not been amended since it wa...
1180,Existing law establishes the State Department ...,This bill requires the State Department of Pub...
683,Existing law provides that the spouse of a wom...,Provides that if a woman conceives through ass...


In [18]:
system_summary = Summaries['SystemSummary']
standard_summary = Summaries['GoldSummary']

In [ ]:
scorer1 = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

In [29]:

scorer_all = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Define a function to compute the Rouge scores
def compute_rouge_scores(row):
    system_summary = row['SystemSummary'].lower()
    gold_summary = row['GoldSummary'].lower()
    return scorer_all.score(gold_summary, system_summary)

# Apply the function to each row
rouge_scores = Summaries.apply(compute_rouge_scores, axis=1)

# Create a DataFrame to store averaged scores
averaged_scores = pd.DataFrame(index=['precision', 'recall', 'fmeasure'])

# Iterate through all ROUGE types and metrics to calculate averages
for rouge_type in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']:
    averaged_scores[rouge_type] = [
        rouge_scores.apply(lambda score_dict: score_dict[rouge_type].precision).mean(),
        rouge_scores.apply(lambda score_dict: score_dict[rouge_type].recall).mean(),
        rouge_scores.apply(lambda score_dict: score_dict[rouge_type].fmeasure).mean(),
    ]



In [31]:
averaged_scores = averaged_scores.T 
averaged_scores

,rouge1,rouge2,rougeL,rougeLsum
precision,0.600587,0.287237,0.363468,0.484640
recall,0.358241,0.159598,0.209563,0.279258
fmeasure,0.407983,0.186211,0.239405,0.321878


In [27]:
scores = {}
for idx, row in Summaries.iterrows():
    system_sum = row['SystemSummary'].lower()
    gold_sum = row['GoldSummary'].lower()
    scores[idx] = scorer.score(system_sum, gold_sum)

In [28]:
scores

{0: {'rouge1': Score(precision=0.14814814814814814, recall=0.6086956521739131, fmeasure=0.23829787234042552),
  'rougeL': Score(precision=0.07936507936507936, recall=0.32608695652173914, fmeasure=0.1276595744680851)},
 1: {'rouge1': Score(precision=0.4366812227074236, recall=0.7299270072992701, fmeasure=0.5464480874316939),
  'rougeL': Score(precision=0.3318777292576419, recall=0.5547445255474452, fmeasure=0.4153005464480874)},
 2: {'rouge1': Score(precision=0.208955223880597, recall=0.358974358974359, fmeasure=0.2641509433962264),
  'rougeL': Score(precision=0.11940298507462686, recall=0.20512820512820512, fmeasure=0.15094339622641506)},
 3: {'rouge1': Score(precision=0.2907488986784141, recall=0.5945945945945946, fmeasure=0.3905325443786982),
  'rougeL': Score(precision=0.14096916299559473, recall=0.2882882882882883, fmeasure=0.1893491124260355)},
 4: {'rouge1': Score(precision=0.34274193548387094, recall=0.8056872037914692, fmeasure=0.4809052333804809),
  'rougeL': Score(precision=0

In [22]:
system_summary.

"expresses the sense of the california legislature that all congressionally chartered veterans' halls and organizations should be exempt from state and federal taxation. makes conforming amendments to the internal revenue code and related federal regulations with respect to the tax exemption for veterans' halls and organizations."

In [ ]:
pd.DataFrame(scores).set_index([['precision','recall','fmeasure']])

In [ ]:
scores1 = scorer1.score(system_summary, standard_summary)

In [ ]:
pd.DataFrame(scores1).set_index([['precision','recall','fmeasure']])